# Laboratorio de regresión logística

|                |   |
:----------------|---|
| **Nombre**     | Camila Garcia Ornelas  |
| **Fecha**      | 12 de octubre, 2022  |
| **Expediente** | 749725  | 

In machine learning, Support Vector Machines (SVM) are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. It is mostly used in classification problems. In this algorithm, each data item is plotted as a point in p-dimensional space (where p is the number of features), with the value of each feature being the value of a particular coordinate. Then, classification is performed by finding the hyper-plane that best differentiates the two classes (or more if we have a multi class problem):

$$ f(x) = w^T \varphi(x) + b $$

where $\varphi: X \rightarrow F $ is a function that makes each input point $x$ correspond to a point in F, where F is a Hilbert space.

In addition to performing linear classification, SVMs can efficiently perform a non-linear classification, implicitly mapping their inputs into high-dimensional feature spaces (more specifically using the kernel trick, like the RBF funcion). 

[1]

OLS utilizes the squared residuals to fit the parameters. Large residuals caused by outliers may worsen the accuracy significantly.

Support Vectors use piecewise linear functions to counter this, in which a hyperparameter  $\epsilon$ called the margin lets errors that are less or equal to it be 0, and error larger than it be $e - \epsilon$. 

The problem to solve is:

\begin{split}
        \min_{w, b, \xi, \xi^*} \mathcal{P}_\epsilon(w, b, \xi) &= \frac{1}{2} w^T w + c \sum_{k=1}^{N} \xi_k \\
        \text{s.t. } & y_k [w^T \varphi(x_k) - b] \geq 1- \xi_k,\ \ k = 1, ..., N \\
        & \xi_k \geq 0,\ \ k = 1, ..., N
\end{split}


The most important question that arises when using a SVM is how to choose the correct hyperplane. Consider the following scenarios:

### Scenario 1

In this scenario there are three hyperplanes called A, B, and C. Now, the problem is to identify the hyperplane which best differentiates the stars and the circles.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/SVM_21-2.png" alt="what image shows"></center>

In this case, hyperplane B separates the stars and the circle betters, hence it is the correct hyperplane.


### Scenario 2

Now take another scenario where all three hyperplanes are segregating classes well. The question that arises is how to choose the best hyperplane in this situation.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/SVM_4-2.png" alt="what image shows"></center>

In such scenarios, we calculate the margin (which is the distance between nearest data point and the hyperplane). The hyperplane with the largest margin will be considered as the correct hyperplane to classify the dataset.

Here C has the largest margin. Hence, it is considered as the best hyperplane.


### Kernels
Knowing 
$$ w = \sum_{k=1}^{N} \alpha_k y_k \varphi(x_k) $$

And
$$ y_{pred} = w^T \varphi(x) + b $$

Then 
$$ y_{pred} = (\sum_{k=1}^{N} \alpha_k y_k \varphi(x_k))^T \varphi(x) + b $$

Where $\varphi$ is a function that makes each input in $x$ correspond to a point in $F$ (a Hilbert space). This can be seen as processing and transforming the input featuers to keep the model's convexity. [2]

This also allows us to transform the inputs into another space where they might be more easily classified.

<center><img src=https://miro.medium.com/max/838/1*gXvhD4IomaC9Jb37tzDUVg.png alt="what image shows"></center>

## ROC and AUC

A ROC (Receiver Operating Characteristic) is a graph that shows how the classification model performs at the classification thresholds. 

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis. This means that the top left corner of the plot is the “ideal” point - a false positive rate of zero, and a true positive rate of one. This is not very realistic, but it does mean that a larger area under the curve (AUC) is usually better. [3]

True Positive Rate is a synonym for Recall and defined as:
$$ TPR = \frac{TP}{TP + FN} $$

False Positive Rate is a synonym for Specificity and defined as:

$$ FPR = \frac{FP}{FP + TN} $$

ROC curves are typically used in binary classification to study the output of a classifier. In order to extend ROC curve and ROC area to multi-label classification, it is necessary to binarize the output. One ROC curve can be drawn per label, but one can also draw a ROC curve by considering each element of the label indicator matrix as a binary prediction (micro-averaging).

E.g. If you lower a classification threshold, more items would be classified as positive, increasing False Positives and True Positives.

AUC stands for Area under the ROC.

## Ejercicio 1

- Utiliza el dataset `Iris`, modela con SVC y haz Cross-Validation de diferentes kernels ('linear', 'poly', 'rbf', 'sigmoid').
- Modela con LogisticRegression.
- El método de Cross-Validation es K-Folds con $k=10$.
- Utiliza el AUC como métrico de Cross-Validation.
- Compara resultados.

In [102]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [103]:
pd.set_option("display.precision", 6)

iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

kf10 = KFold(n_splits=10, shuffle=True, random_state=42)

resultados_iris = []

for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    svc = SVC(kernel=kernel, probability=True, random_state=42)
    pipe = Pipeline([('scaler', StandardScaler()), ('clf', svc)])
    scores = cross_val_score(pipe, X_iris, y_iris, cv=kf10, scoring='roc_auc_ovr')
    resultados_iris.append(
        dict([
            ('Modelo', 'SVC (' + kernel + ')'),
            ('AUC media (10-Fold)', scores.mean()),
            ('AUC std', scores.std())
        ])
    )

In [104]:
logreg = LogisticRegression(max_iter=200,random_state=137)
pipe_lr = Pipeline([('scaler', StandardScaler()), ('clf', logreg)])
scores_lr = cross_val_score(pipe_lr, X_iris, y_iris, cv=kf10, scoring='roc_auc_ovr')
resultados_iris.append(
    dict([
        ('Modelo', 'LogisticRegression'),
        ('AUC media (10-Fold)', scores_lr.mean()),
        ('AUC std', scores_lr.std())
    ])
)

df_iris = pd.DataFrame(resultados_iris).sort_values('AUC media (10-Fold)', ascending=False).reset_index(drop=True)
df_iris

,Modelo,AUC media (10-Fold),AUC std
0,SVC (linear),1.000000,0.000000
1,SVC (poly),1.000000,0.000000
2,LogisticRegression,0.996655,0.008244
3,SVC (rbf),0.995875,0.012374
4,SVC (sigmoid),0.981926,0.033252


Los modelos dieron métricas idénticas salvo el sigmoid. En consecuencia, cualquiera funciona bien excepto el sigmoid.

## Ejercicio 2
- Repite el ejercicio 1 con el dataset `Default`. Utiliza `default` como target.

In [107]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

pd.set_option("display.precision", 6)

ruta = "Default.csv"
df = pd.read_csv(ruta)

df.columns = [c.strip().lower() for c in df.columns]

col_target = "default"
col_balance = "balance"
col_income  = "income"
col_student = "student" if "student" in df.columns else None

if col_student is not None:
    serie_std = df[col_student].astype(str).str.strip().str.lower()
    df[col_student] = (serie_std == "yes").astype(int)

y_series = df[col_target].astype(str).str.strip().str.lower()
y = (y_series == "yes").astype(int).values

cols_X = [c for c in [col_balance, col_income, col_student] if c is not None and c in df.columns]
X = df[cols_X].apply(pd.to_numeric, errors="coerce").fillna(0.0).values

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

print("Columnas usadas en X:", cols_X)
print("Tamaño X:", X.shape, "   Tamaño y:", y.shape)

Columnas usadas en X: ['balance', 'income', 'student']
Tamaño X: (10000, 3)    Tamaño y: (10000,)


In [ ]:
resultados = []

for kernel in ["linear", "poly", "rbf", "sigmoid"]:
    modelo = SVC(kernel=kernel, probability=True, random_state=42)
    pipe = Pipeline([("scaler", StandardScaler()), ("clf", modelo)])
    auc_scores = cross_val_score(pipe, X, y, cv=skf, scoring="roc_auc")
    fila = ["SVC (" + kernel + ")", auc_scores.mean(), auc_scores.std()]
    resultados.append(fila)

df_svc = pd.DataFrame(resultados, columns=["Modelo", "AUC media (10-Fold)", "AUC std"])
df_svc

In [ ]:
logreg = LogisticRegression(max_iter=1000)
pipe_lr = Pipeline([("scaler", StandardScaler()), ("clf", logreg)])
scores_lr = cross_val_score(pipe_lr, X, y, cv=skf, scoring="roc_auc")

fila_lr = ["LogisticRegression", scores_lr.mean(), scores_lr.std()]
tabla = pd.concat([df_svc, pd.DataFrame([fila_lr], columns=df_svc.columns)], ignore_index=True)
tabla = tabla.sort_values("AUC media (10-Fold)", ascending=False).reset_index(drop=True)
tabla

Con el nuevo resultado, sigmoid continúa al final. El mejor SVC fue el lineal (0.9233), pero la logística quedó arriba con 0.9493.

# Addendum

Métricos disponibles para clasificación:
- ‘accuracy’
- ‘balanced_accuracy’
- ‘top_k_accuracy’
- ‘average_precision’
- ‘neg_brier_score’
- ‘f1’
- ‘f1_micro’
- ‘f1_macro’
- ‘f1_weighted’
- ‘f1_samples’
- ‘neg_log_loss’
- ‘precision’ etc.
- ‘recall’ etc.
- ‘jaccard’ etc.
- ‘roc_auc’
- ‘roc_auc_ovr’
- ‘roc_auc_ovo’
- ‘roc_auc_ovr_weighted’
- ‘roc_auc_ovo_weighted’
- ‘d2_log_loss_score’

# References

[1] Shigeo Abe.Support Vector Machines for Pattern Classification,2Ed.Springer-Verlag London,2010. ISBN978-1-84996-097-7. URLhttps://www.springer.com/gp/book/9781849960977.

[2] Johan A K Suykens, Tony Van Gestel, Jos De Brabanter, BartDe Moor, and Joos Vandewalle.Least Squares Support VectorMachines. World Scientific,2002. ISBN9789812381514. URLhttps://www.worldscientific.com/worldscibooks/10.1142/5089.

[3] Bradley, A. P. (1997). The use of the area under the ROC curve in the evaluation of machine learning algorithms. Pattern recognition, 30(7), 1145-1159. URL https://www.researchgate.net/post/how_can_I_interpret_the_ROC_curve_result